# Stack Overflow


## SageMaker Setup

In [1]:
!pip install transformers datasets ipywidgets IProgress sagemaker -qq -U

In [2]:
!mkdir dataset

mkdir: cannot create directory ‘dataset’: File exists


In [3]:
import sagemaker

sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

role = sagemaker.get_execution_role()
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sagemaker_session_bucket}")
print(f"sagemaker session region: {sess.boto_region_name}")


sagemaker role arn: arn:aws:iam::123456789012:role/service-role/AmazonSageMaker-ExecutionRole-20180314T175147
sagemaker bucket: sagemaker-us-east-1-123456789012
sagemaker session region: us-east-1


#### Define model name

In [4]:
model_name = 'bert-base-uncased'

In [5]:
from datasets import load_dataset, Dataset # TR NLP
import pandas as pd
import numpy as np
import transformers
from sklearn import metrics


from transformers import AutoTokenizer, BertTokenizer, BertModel, BertConfig
import torch
from torch.utils.data import Dataset, DataLoader # MLP , RandomSampler, SequentialSampler

from sagemaker.huggingface.model import HuggingFacePredictor

pd.set_option('display.max_colwidth', 0)

In [6]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cuda'

In [7]:
df = pd.read_csv('softrain5Kv2.csv')

In [8]:
df.head(2)

,labels,source
0,Application Development and Application Integration,how to read terraform list from vars fileI have a list of subnets as list in the vars file; which I want to read a list inside the AWS resource.variables.tfvariable &quot;alb_subnets&quot; { type = list(string)}terraform.tfvarssubnets = [&quot;subnet-a&quot;;&quot;subnet-b&quot;]main.tfresource &quot;&lt;resource_name&gt;&quot; &quot;test&quot; { name = var.Name security_groups = [&quot;${join(&quot;;&quot;;var.subnets)}&quot;]}Error: ValidationError: subnet 'subnet-a;subnet-b' is not valid status code: 400; request id: dc4be07e-e353-4821-8bde-3d9849584befwhat is the right way to read the list as list inside the AWS resource.
1,Networking|Other,How to execute a command in (from) zabbix aws elbv2How to execute a command in (from) zabbix UserParameter=alb.co[*];aws elbv2 describe-listener-certificates --listener-arn ******* in sudores.d in file zabbix Defaults:zabbix !requiretty zabbix ALL=NOPASSWD: ALLthe command is not executed.Added credential (macros) to Template&lt;zabbix_export&gt; &lt;version&gt;5.0&lt;/version&gt; &lt;date&gt;2020-07-24T10:52:10Z&lt;/date&gt; &lt;groups&gt; &lt;group&gt; &lt;name&gt;Templates&lt;/name&gt; &lt;/group&gt; &lt;/groups&gt; &lt;templates&gt; &lt;template&gt; &lt;template&gt;Template alb-co&lt;/template&gt; &lt;name&gt;Template ssl-count&lt;/name&gt; &lt;groups&gt; &lt;group&gt; &lt;name&gt;Templates&lt;/name&gt; &lt;/group&gt; &lt;/groups&gt; &lt;applications&gt; &lt;application&gt; &lt;name&gt;alb-co&lt;/name&gt; &lt;/application&gt; &lt;/applications&gt; &lt;items&gt; &lt;item&gt; &lt;name&gt;alb-co&lt;/name&gt; &lt;key&gt;alb.co[arn;{$HOST.DNS};--access-key{$AWS_ACCESS_KEY};--secret-key{$AWS_SECRET_KEY};--region{$REGION}]&lt;/key&gt; &lt;delay&gt;30s&lt;/delay&gt; &lt;applications&gt; &lt;application&gt; &lt;name&gt;alb-co&lt;/name&gt; &lt;/application&gt; &lt;/applications&gt; &lt;triggers&gt; &lt;trigger&gt; &lt;expression&gt;{last()}&amp;gt;=23&lt;/expression&gt; &lt;name&gt;{ITEM.LASTVALUE} alb-co {HOST.DNS}&lt;/name&gt; &lt;priority&gt;INFO&lt;/priority&gt; &lt;manual_close&gt;YES&lt;/manual_close&gt;&lt;/trigger&gt; &lt;/triggers&gt; &lt;/item&gt; &lt;/items&gt; &lt;macros&gt; &lt;macro&gt; &lt;macro&gt;{$AWS_ACCESS_KEY}&lt;/macro&gt; &lt;type&gt;SECRET_TEXT&lt;/type&gt; &lt;/macro&gt; &lt;macro&gt; &lt;macro&gt;{$AWS_SECRET_KEY}&lt;/macro&gt; &lt;type&gt;SECRET_TEXT&lt;/type&gt; &lt;/macro&gt; &lt;macro&gt; &lt;macro&gt;{$REGION}&lt;/macro&gt; &lt;value&gt;eu-central-1&lt;/value&gt; &lt;/macro&gt; &lt;/macros&gt; &lt;/template&gt; &lt;/templates&gt;&lt;/zabbix_export&gt;But still doesn't work.There are no item and trigger errors; it just returns the value 0;but should 26. Maybe someone started with this?


## Prepare Data

In [9]:
df['labels'] = df['labels'].str.split('|')

In [10]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
yt = mlb.fit_transform(df['labels'])
# Getting a sense of how the tags data looks like
print(yt[0])
print(mlb.inverse_transform(yt[0].reshape(1,-1)))
print(mlb.classes_)

[0 1 0 0 0 0 0]
[('Application Development and Application Integration',)]
['Analytics and AI/ML'
 'Application Development and Application Integration' 'Compute'
 'Database and Storage' 'Governance, Security and Ops' 'Networking'
 'Other']


In [11]:
dfy = pd.DataFrame(yt)

In [12]:
dfy['combined']= dfy.values.tolist()

In [13]:
# dfy

In [14]:
df['ymlblabels'] = dfy['combined']

In [15]:
df.drop(['labels'], axis=1, inplace=True)
df =  df.rename(columns={"source": "comment_text", "ymlblabels": "target_classification"})

In [16]:
df.head(2)

,comment_text,target_classification
0,how to read terraform list from vars fileI have a list of subnets as list in the vars file; which I want to read a list inside the AWS resource.variables.tfvariable &quot;alb_subnets&quot; { type = list(string)}terraform.tfvarssubnets = [&quot;subnet-a&quot;;&quot;subnet-b&quot;]main.tfresource &quot;&lt;resource_name&gt;&quot; &quot;test&quot; { name = var.Name security_groups = [&quot;${join(&quot;;&quot;;var.subnets)}&quot;]}Error: ValidationError: subnet 'subnet-a;subnet-b' is not valid status code: 400; request id: dc4be07e-e353-4821-8bde-3d9849584befwhat is the right way to read the list as list inside the AWS resource.,"[0, 1, 0, 0, 0, 0, 0]"
1,How to execute a command in (from) zabbix aws elbv2How to execute a command in (from) zabbix UserParameter=alb.co[*];aws elbv2 describe-listener-certificates --listener-arn ******* in sudores.d in file zabbix Defaults:zabbix !requiretty zabbix ALL=NOPASSWD: ALLthe command is not executed.Added credential (macros) to Template&lt;zabbix_export&gt; &lt;version&gt;5.0&lt;/version&gt; &lt;date&gt;2020-07-24T10:52:10Z&lt;/date&gt; &lt;groups&gt; &lt;group&gt; &lt;name&gt;Templates&lt;/name&gt; &lt;/group&gt; &lt;/groups&gt; &lt;templates&gt; &lt;template&gt; &lt;template&gt;Template alb-co&lt;/template&gt; &lt;name&gt;Template ssl-count&lt;/name&gt; &lt;groups&gt; &lt;group&gt; &lt;name&gt;Templates&lt;/name&gt; &lt;/group&gt; &lt;/groups&gt; &lt;applications&gt; &lt;application&gt; &lt;name&gt;alb-co&lt;/name&gt; &lt;/application&gt; &lt;/applications&gt; &lt;items&gt; &lt;item&gt; &lt;name&gt;alb-co&lt;/name&gt; &lt;key&gt;alb.co[arn;{$HOST.DNS};--access-key{$AWS_ACCESS_KEY};--secret-key{$AWS_SECRET_KEY};--region{$REGION}]&lt;/key&gt; &lt;delay&gt;30s&lt;/delay&gt; &lt;applications&gt; &lt;application&gt; &lt;name&gt;alb-co&lt;/name&gt; &lt;/application&gt; &lt;/applications&gt; &lt;triggers&gt; &lt;trigger&gt; &lt;expression&gt;{last()}&amp;gt;=23&lt;/expression&gt; &lt;name&gt;{ITEM.LASTVALUE} alb-co {HOST.DNS}&lt;/name&gt; &lt;priority&gt;INFO&lt;/priority&gt; &lt;manual_close&gt;YES&lt;/manual_close&gt;&lt;/trigger&gt; &lt;/triggers&gt; &lt;/item&gt; &lt;/items&gt; &lt;macros&gt; &lt;macro&gt; &lt;macro&gt;{$AWS_ACCESS_KEY}&lt;/macro&gt; &lt;type&gt;SECRET_TEXT&lt;/type&gt; &lt;/macro&gt; &lt;macro&gt; &lt;macro&gt;{$AWS_SECRET_KEY}&lt;/macro&gt; &lt;type&gt;SECRET_TEXT&lt;/type&gt; &lt;/macro&gt; &lt;macro&gt; &lt;macro&gt;{$REGION}&lt;/macro&gt; &lt;value&gt;eu-central-1&lt;/value&gt; &lt;/macro&gt; &lt;/macros&gt; &lt;/template&gt; &lt;/templates&gt;&lt;/zabbix_export&gt;But still doesn't work.There are no item and trigger errors; it just returns the value 0;but should 26. Maybe someone started with this?,"[0, 0, 0, 0, 0, 1, 1]"


In [17]:
df.to_csv('softrain5Ktrain.csv', encoding='utf-8', index=False)

## Custom Dataset
https://github.com/abhimishra91/transformers-tutorials/blob/master/transformers_multi_label_classification.ipynb


In [34]:
# Sections of config

# Defining some key variables that will be used later on in the training
MAX_LEN = 200
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 4
EPOCHS = 10
LEARNING_RATE = 1e-05


In [35]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [36]:
class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.comment_text = dataframe.comment_text
        self.targets = self.data.target_classification
        self.max_len = max_len
#         print('HELLOOOOOO INIT')

    def __len__(self):
        return len(self.comment_text)
    
    def data(self):
        return self._data


    def __getitem__(self, index):
        comment_text = str(self.comment_text[index])
#         print(comment_text)
        comment_text = " ".join(comment_text.split())
 
        inputs = self.tokenizer.encode_plus(
            comment_text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }


In [37]:
# Creating the dataset and dataloader for the neural network

train_size = 0.9
train_dataset=df.sample(frac=train_size,random_state=200)
test_dataset=df.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)


print("FULL Dataset: {}".format(df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = CustomDataset(train_dataset, tokenizer, MAX_LEN)
testing_set = CustomDataset(test_dataset, tokenizer, MAX_LEN)

FULL Dataset: (5000, 2)
TRAIN Dataset: (4500, 2)
TEST Dataset: (500, 2)


In [38]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

## Custom NLP Model

In [39]:
# return_dict=False https://stackoverflow.com/questions/65082243/dropout-argument-input-position-1-must-be-tensor-not-str-when-using-bert

In [40]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model. 

class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.l1 = transformers.BertModel.from_pretrained('bert-base-uncased', return_dict=False)
        self.l2 = torch.nn.Dropout(0.3)
        self.l3 = torch.nn.Linear(768, 7)
    
    def forward(self, ids, mask, token_type_ids):
        _, output_1= self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids)
        output_2 = self.l2(output_1)
        output = self.l3(output_2)
        return output

model = BERTClass()
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BERTClass(
  (l1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    

In [41]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [42]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [43]:
def train(epoch):
    model.train()
    for _,data in enumerate(training_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)
        
#         print(f"\n======================\n\nIDs-----\n{ids}\n-----MASK-----\n{mask}\n-----\nTOKEN TYPE IDs----{token_type_ids}\n\n")

#         print("\n *********** SHAPE ids {}\n".format(ids.shape))
#         print("\n *********** SHAPE mask {}\n".format(mask.shape))
#         print("\n *********** SHAPE token_type_ids {}\n".format(token_type_ids.shape))
        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        if _%100==0:
            print(f'Epoch: {epoch}, Batch: {_}, Loss:  {loss.item()}')
#         print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


In [44]:
for epoch in range(EPOCHS):
    train(epoch)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/opt/conda/lib/python3.6/site-packages/transformers/tokenization_utils_base.py:2232: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Epoch: 0, Batch: 0, Loss:  0.7606750726699829
Epoch: 0, Batch: 100, Loss:  0.5396038889884949
Epoch: 0, Batch: 200, Loss:  0.5140712261199951
Epoch: 0, Batch: 300, Loss:  0.5634191036224365
Epoch: 0, Batch: 400, Loss:  0.4400113821029663
Epoch: 0, Batch: 500, Loss:  0.37731286883354187
Epoch: 1, Batch: 0, Loss:  0.3461363911628723
Epoch: 1, Batch: 100, Loss:  0.3563036322593689
Epoch: 1, Batch: 200, Loss:  0.32605138421058655
Epoch: 1, Batch: 300, Loss:  0.37069427967071533
Epoch: 1, Batch: 400, Loss:  0.41086670756340027
Epoch: 1, Batch: 500, Loss:  0.3334399163722992
Epoch: 2, Batch: 0, Loss:  0.292452871799469
Epoch: 2, Batch: 100, Loss:  0.29512399435043335
Epoch: 2, Batch: 200, Loss:  0.1622379720211029
Epoch: 2, Batch: 300, Loss:  0.25359535217285156
Epoch: 2, Batch: 400, Loss:  0.2948223054409027
Epoch: 2, Batch: 500, Loss:  0.29760852456092834
Epoch: 3, Batch: 0, Loss:  0.23609690368175507
Epoch: 3, Batch: 100, Loss:  0.15604087710380554
Epoch: 3, Batch: 200, Loss:  0.229609608

## Validating the Model

In [91]:
def validation(epoch):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, data in enumerate(testing_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets, ids

About model scoring, multi label: https://scikit-learn.org/stable/modules/model_evaluation.html

In [92]:
# for epoch in range(EPOCHS):
#     outputs, targets = validation(epoch)
# #     print(targets)
# #     print(outputs)
#     outputs = np.array(outputs) >= 0.5
#     accuracy = metrics.accuracy_score(targets, outputs)
#     f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
#     f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
#     f1_score_weighted = metrics.f1_score(targets, outputs, average='weighted')
#     f1_score_sample = metrics.f1_score(targets, outputs, average='samples')
#     f1_score_none = metrics.f1_score(targets, outputs, average=None)
#     precision = metrics.precision_score(targets, outputs, average='samples')
#     recall = metrics.recall_score(targets, outputs, average='samples')

#     print(f"Accuracy Score = {accuracy}")
#     print(f"F1 Score (Micro) = {f1_score_micro}")
#     print(f"F1 Score (Macro) = {f1_score_macro}")
#     print(f"F1 Score (Weighted) = {f1_score_weighted}")
#     print(f"F1 Score (Sample) = {f1_score_sample}")
#     print(f"F1 Score (None) = {f1_score_none}")
#     print('[0: Analytics and AI/ML 1:Application Development and Application Integration 2:Compute 3: Database and Storage 4: Governance Security and Ops 5: Networking 6: Other]')
#     print(f"Precision Score (Samples) = {precision}")
#     print(f"Recall Score (Samples) = {recall}")


In [94]:
# for epoch in range(EPOCHS):
outputs, targets, comment_ids = validation(epoch)
#     print(targets)
#     print(outputs)
outputs = np.array(outputs) >= 0.5
accuracy = metrics.accuracy_score(targets, outputs)
f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
f1_score_weighted = metrics.f1_score(targets, outputs, average='weighted')
f1_score_sample = metrics.f1_score(targets, outputs, average='samples')
f1_score_none = metrics.f1_score(targets, outputs, average=None)
precision = metrics.precision_score(targets, outputs, average='samples')
recall = metrics.recall_score(targets, outputs, average='samples')

print(f"Accuracy Score = {accuracy}")
print(f"F1 Score (Micro) = {f1_score_micro}")
print(f"F1 Score (Macro) = {f1_score_macro}")
print(f"F1 Score (Weighted) = {f1_score_weighted}")
print(f"F1 Score (Sample) = {f1_score_sample}")
print(f"F1 Score (None) = {f1_score_none}")
print('[0: Analytics and AI/ML 1:Application Development and Application Integration 2:Compute 3: Database and Storage 4: Governance Security and Ops 5: Networking 6: Other]')
print(f"Precision Score (Samples) = {precision}")
print(f"Recall Score (Samples) = {recall}")


Accuracy Score = 0.338
F1 Score (Micro) = 0.678809957498482
F1 Score (Macro) = 0.664171382585759
F1 Score (Weighted) = 0.6706835647203292
F1 Score (Sample) = 0.6856666666666668
F1 Score (None) = [0.78571429 0.61096606 0.75625    0.80808081 0.60223048 0.70063694
 0.3853211 ]
[0: Analytics and AI/ML 1:Application Development and Application Integration 2:Compute 3: Database and Storage 4: Governance Security and Ops 5: Networking 6: Other]
Precision Score (Samples) = 0.7032333333333334
Recall Score (Samples) = 0.736


In [95]:
from sklearn.metrics import classification_report

target_names = ['Analytics and AI/ML', 'Application Development and Application Integration', 'Compute', 
                'Database and Storage', 'Governance, Security and Ops', 'Networking', 'Other']
print(classification_report(targets, outputs, target_names=target_names))

                                                     precision    recall  f1-score   support

                                Analytics and AI/ML       0.80      0.77      0.79        57
Application Development and Application Integration       0.59      0.64      0.61       184
                                            Compute       0.68      0.85      0.76       143
                               Database and Storage       0.75      0.88      0.81       137
                       Governance, Security and Ops       0.55      0.67      0.60       121
                                         Networking       0.74      0.66      0.70        83
                                              Other       0.58      0.29      0.39        73

                                          micro avg       0.66      0.70      0.68       798
                                          macro avg       0.67      0.68      0.66       798
                                       weighted avg       0.66      

In [97]:
comment_ids[:1]

tensor([[  101,  2003,  2045,  5724,  2000,  2417,  7442,  6593,  8299,  4026,
          2000, 16770,  1999, 22091,  2015,  2897,  7170,  5703,  6657,  4438,
          7170,  5703,  2099,  1006, 18856,  2497,  1007,  1998,  4646,  7170,
          5703,  2099,  1006,  2632,  2497,  1007,  2045,  2003,  5724,  2000,
          2417,  7442,  6593,  2035,  8299,  4026,  2000, 16770, 19373,  1012,
          1045,  2079,  2025,  2424,  1996,  5724,  2000,  2417,  7442,  6593,
         22975,  2361,  3417,  3770,  4026,  2000,  1056,  4877,  3417,  4008,
          2509,  2013, 17953,  2497,  1006,  2897,  7170,  5703,  2099,  1007,
          2151,  2393,  2003,  2172, 12315,  1012,   102,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,  

In [51]:
targets[:5]

[[0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0],
 [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
 [0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]]

In [52]:
outputs[:5]

array([[False,  True,  True, False, False, False, False],
       [False,  True, False, False,  True, False, False],
       [False,  True, False,  True, False, False, False],
       [False, False, False, False,  True, False, False],
       [False, False, False,  True, False, False, False]])

In [98]:
# testing_loader

https://huggingface.co/docs/datasets/quicktour.html

In [99]:
# next(iter(testing_loader))

In [100]:
# testing_set[0]

## Visualize Dataset

In [104]:

for i in range(len(comment_ids)):
    print(f"Sample ----- {i}")
    print(tokenizer.decode(comment_ids[i]))
    print("Data: Targets", targets[i])
    print("Data: Outputs", outputs[i])

Sample ----- 0
[CLS] is there option to redirect http traffic to https in aws network load balancerin classic load balancer ( clb ) and application load balancer ( alb ) there is option to redirect all http traffic to https listener. i do not find the option to redirect tcp port 80 traffic to tls port 443 from nlb ( network load balancer ) any help is much appreciated. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD

In [90]:
i = 0
for datax in testing_set:
#   print("Data: ", datax) # ids, mask, token_type_ids, targets
  print("Data IDs ", datax['ids'])
#   print("Waveform: {}\nSample rate: \nLabels: ".format(datax[0] )) # , datax[1])) # , datax[2]))
  print(tokenizer.decode(datax['ids']))
  print("Data: Targets", datax['targets'])

#   print("Data: ", datax['ids'][2])

  if i == 1:
      break
  i = i + 1

Data IDs  tensor([  101,  2025,  2583,  2000,  9699,  9230,  2015,  1999,  8946,  2121,
         6520, 23422,  4646,  2072,  2572,  2667,  2000,  2448,  9230,  2015,
         1999,  1996,  6520, 23422,  4646,  2008,  2003,  2478,  8946,  2121,
         2021,  1996,  3431,  2024,  2025, 10842,  1012,  1045,  2018,  2053,
         3277,  2007,  1996,  3988,  9230,  2015,  2021,  2044,  2008,  1025,
         2065,  1045,  2191,  2151,  3431,  2000,  2026,  4275,  2008,  2024,
         2025,  7686,  1999,  1996,  9230,  2015,  1998,  2009,  7906, 11717,
         3988,  9230,  2015,  1012,  1045,  2572,  2025,  2583,  2000,  2424,
         2151,  9230,  5371,  2205,  2672,  2008,  2003,  4786,  2023,  3291,
         2021,  1045,  2572,  2025,  2583,  2000,  2424,  2151,  5576,  1012,
         1045,  2572,  2478,  9733, 16428,  2015,  2004,  7809,  1998,  4646,
         2003,  2036,  4354,  2006, 22091,  2015,  1998,  1045,  2572,  2478,
         2023,  3094,  2005,  9230,  2015,  1012,  894

In [79]:
# from tqdm import tqdm
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
# model.train().to(device)
# optimizer = torch.optim.AdamW(params=model.parameters(), lr=1e-5)
# for epoch in range(1):
#     for i, batch in enumerate(tqdm(training_loader)):
#         batch = {k: v.to(device) for k, v in batch.items()}
#         outputs = model(**batch)
#         loss = outputs[0]
#         loss.backward()
#         optimizer.step()
#         optimizer.zero_grad()
#         if i % 10 == 0:
#             print(f"loss: {loss}")

In [76]:
i = 0
for datax in testing_loader:
#   print("Data: ", datax) # ids, mask, token_type_ids, targets
  print("Data IDs ", datax['ids'][0])
#   print("Waveform: {}\nSample rate: \nLabels: ".format(datax[0] )) # , datax[1])) # , datax[2]))
  print(tokenizer.decode(datax['ids'][0]))
  print("Data: Targets", datax['targets'][0])

#   print("Data: ", datax['ids'][2])

  if i == 1:
      break
  i = i + 1

Data IDs  tensor([  101, 22091,  2015, 15775, 13231,  2334,  2573,  2021,  2025, 15775,
        13231, 21296,  2072,  2572,  3492,  2047,  2000, 16861,  1998, 22091,
         2015, 15775, 13231,  1012,  1045,  2699,  3015,  1037,  3642,  2008,
         4152,  7696,  2013,  6202,  1011,  3193,  1998, 15389,  2015,  4449,
         5834,  2006,  1996,  7755,  1012,  1045,  7718,  1996,  3642,  7246,
         1998,  2673,  2499,  2986,  1025,  2021,  2043,  1045,  3231,  1996,
         2717, 17928,  1045,  2131,  1996,  2206,  7561,  1024,  1063,  1004,
        22035,  2102,  1025,  4471,  1004, 22035,  2102,  1025,  1024,  1004,
        22035,  2102,  1025,  4394, 27280, 19204,  1004, 22035,  2102,  1025,
         1065,  1045,  2275,  2039,  2026, 22496,  3081,  1004, 22035,  2102,
         1025, 22091,  2015,  9530,  8873, 27390,  2063,  1004, 22035,  2102,
         1025,  2004,  4541,  2182,  1024, 16770,  1024,  1013,  1013,  9986,
         2015,  1012, 22091,  2015,  1012,  9733,  101

In [48]:
--STOP--

SyntaxError: invalid syntax (<ipython-input-48-f1d5dff74306>, line 1)

## Load Dataset

In [ ]:
# dataset = Dataset.from_pandas(df)

In [ ]:
# dataset

In [ ]:
# Download dataset
from datasets import load_dataset
dataset = load_dataset('csv', data_files='softrain5Ktrain.csv')

In [ ]:
dataset

In [ ]:
train_test = dataset['train'].train_test_split(test_size=0.1)

In [ ]:
train_test

In [ ]:
train_dataset = train_test['train']
test_dataset = train_test['test']

Now we can inspect the training data

In [ ]:
df_train = pd.DataFrame(train_dataset)

In [ ]:
pd.set_option('display.max_colwidth', 0)

In [ ]:
df_train.head(5)

Before we can start the training we need to tokenize the data save it in S3

https://discuss.huggingface.co/t/fine-tune-for-multiclass-or-multilabel-multiclass/4035/2

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

# tokenizer helper function
def tokenize(batch):
    return tokenizer(batch['ymlblabels'], padding='max_length', truncation=True)

In [ ]:
train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

In [ ]:
train_dataset =  train_dataset.rename_column("ymlblabels", "labels")
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset = test_dataset.rename_column("ymlblabels", "labels")
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])

In [ ]:
s3_prefix_sentiment = 'datasets/stackoverflow-nlp-hf'

In [ ]:
import botocore
from datasets.filesystems import S3FileSystem

s3 = S3FileSystem()  

# save train_dataset to s3
training_input_path = f's3://{sagemaker_session_bucket}/{s3_prefix_sentiment}/train'
train_dataset.save_to_disk(training_input_path,fs=s3)

# save test_dataset to s3
test_input_path = f's3://{sagemaker_session_bucket}/{s3_prefix_sentiment}/test'
test_dataset.save_to_disk(test_input_path,fs=s3)

### Model Training

In [ ]:
from sagemaker.huggingface import HuggingFace

# hyperparameters, which are passed into the training job
hyperparameters_sentiment={'epochs': 1,
                 'train_batch_size': 8,
                 'model_name': model_name
                 }

In [ ]:
huggingface_estimator_sentiment = HuggingFace(entry_point='train.py',
                                    source_dir='./scripts',
                                    instance_type='ml.p3.2xlarge',
                                    instance_count=1,
                                    role=role,
                                    transformers_version='4.6',
                                    pytorch_version='1.7',
                                    py_version='py36',
                                    hyperparameters=hyperparameters_sentiment,
                                    )

In [ ]:
huggingface_estimator_sentiment.fit({'train': training_input_path, 'test': test_input_path}, 
                                    wait=False)

https://medium.com/analytics-vidhya/multi-label-text-classification-using-transformers-bert-93460838e62b

### Model Deployment

In [ ]:
predictor_sentiment = huggingface_estimator_sentiment.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.xlarge",
    wait=False,
    endpoint_name="turkish-sentiment-endpoint"
)

### Model Testing

In [ ]:
# This is only required to create a predictor from an already deployed model
predictor_sentiment = HuggingFacePredictor('turkish-sentiment-endpoint')

In [ ]:
# Input text: "This is a pretty bad product, I wouldn't recommend this to anyone"
sentiment_input= {"inputs": "Bu oldukça kötü bir ürün, bunu kimseye tavsiye etmem"}
predictor_sentiment.predict(sentiment_input)

In [ ]:
#Input text: "I love this shampoo, it makes my hair so shiny"
sentiment_input= {"inputs": "Bu şampuanı seviyorum, saçlarımı çok parlak yapıyor"}
predictor_sentiment.predict(sentiment_input)